"# کلون ریپازیتوری\n",
    "!git clone https://github.com/aminak58/RL-Trading-Validation.git\n",
    "\n",
    "# رفتن به مسیر پروژه\n",
    "%cd RL-Trading-Validation\n",
    "\n",
    "# بررسی فایل‌ها\n",
    "!ls -la\n",
    "!echo \"\\n=== محتویات user_data ===\"\n",
    "!ls -la user_data/\n",
    "!echo \"\\n=== داده‌های معاملاتی ===\"\n",
    "!ls -la user_data/data/binance/"

## 🔧 مرحله ۱: کلون ریپازیتوری و آماده‌سازی

In [ ]:
"# نصب Freqtrade و وابستگی‌ها\n",
    "!pip install freqtrade\n",
    "\n",
    "# نصب PyTorch به صورت دستی برای جلوگیری از خطا\n",
    "!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121\n",
    "\n",
    "# نصاب سایر وابستگی‌ها\n",
    "!pip install stable-baselines3>=2.2.0\n",
    "!pip install gymnasium>=0.28.0\n",
    "!pip install datasieve==0.1.9\n",
    "!pip install matplotlib seaborn plotly jupyter\n",
    "\n",
    "print(\"✅ تمام وابستگی‌ها با موفقیت نصب شدند!\")"

## 📦 مرحله ۲: نصب وابستگی‌ها (نسخه‌های سازگار)

In [ ]:
# نصب Freqtrade و وابستگی‌ها
!pip install freqtrade

# نصب نسخه‌های سازگار برای حل مشکل API
!pip install torch==2.0.0 torchvision==0.15.0
!pip install stable-baselines3==2.2.0
!pip install gymnasium==0.28.0
!pip install numpy==1.26.0 pandas==2.0.3
!pip install scikit-learn==1.3.0
!pip install datasieve==0.1.9
!pip install matplotlib seaborn plotly jupyter

print("✅ تمام وابستگی‌ها با نسخه‌های سازگار نصب شدند!")

## 🧪 مرحله ۳: تست محیط و بررسی نسخه‌ها

In [ ]:
import sys
import torch
import stable_baselines3 as sb3
import gymnasium
import numpy as np
import pandas as pd
import freqtrade

print("📋 نسخه‌های نصب شده:")
print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"Stable-Baselines3: {sb3.__version__}")
print(f"Gymnasium: {gymnasium.__version__}")
print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")
print(f"Freqtrade: {freqtrade.__version__}")

# تست API compatibility
print("\n🧪 تست سازگاری API...")
try:
    from stable_baselines3 import PPO
    from gymnasium import spaces
    
    # تست predict function
    class TestEnv:
        def __init__(self):
            self.action_space = spaces.Discrete(5)
            self.observation_space = spaces.Box(low=-1, high=1, shape=(10,))

        def reset(self):
            return np.array([0.0] * 10), {}

        def step(self, action):
            return np.array([0.0] * 10), 0.0, False, False, {}
    
    test_env = TestEnv()
    test_model = PPO("MlpPolicy", test_env, verbose=0)
    test_obs = np.array([0.0] * 10)
    test_action, _ = test_model.predict(test_obs, deterministic=True)
    
    print(f"✅ تست API موفق بود! Action: {test_action}")
    print("🎉 هیچ مشکل API وجود ندارد!")
    
except Exception as e:
    print(f"❌ مشکل API: {e}")
    print("⚠️ نیاز به بررسی وابستگی‌ها دارید")

## 📊 مرحله ۴: بررسی داده‌های معاملاتی

In [ ]:
import os
import json
import pandas as pd

# بررسی داده‌ها
data_dir = "user_data/data/binance"
pairs = ["BTC_USDT_USDT-5m-futures", "ETH_USDT_USDT-5m-futures", 
         "SOL_USDT_USDT-5m-futures", "DOGE_USDT_USDT-5m-futures"]

print("📊 بررسی داده‌های معاملاتی:")
total_data_points = 0

for pair in pairs:
    file_path = os.path.join(data_dir, f"{pair}.json")
    if os.path.exists(file_path):
        # بررسی حجم فایل
        file_size = os.path.getsize(file_path) / (1024*1024)  # MB
        
        # خواندن نمونه داده
        try:
            with open(file_path, 'r') as f:
                sample_data = json.load(f)
                
            print(f"\n📈 {pair}:")
            print(f"   حجم: {file_size:.2f} MB")
            print(f"   تعداد دیتا پوینت‌ها: {len(sample_data):,}")
            
            if len(sample_data) > 0:
                sample_candle = sample_data[0]
                print(f"   نمونه کندل: {sample_candle}")
            
            total_data_points += len(sample_data)
            
        except Exception as e:
            print(f"   ❌ خطا در خواندن {pair}: {e}")
    else:
        print(f"❌ فایل {pair} یافت نشد")

print(f"\n📊 مجموع داده‌ها: {total_data_points:,} دیتا پوینت")
print(f"📅 تقریباً {total_data_points // (12 * 24)} روز داده")

## 🧠 مرحله ۵: آموزش مدل RL

In [ ]:
# آموزش مدل RL برای یک ماه داده
print("🧠 شروع آموزش مدل RL...")
print("⏰ این فرآیند ممکن است 20-30 دقیقه طول بکشد")

# دستور آموزش
train_command = """freqtrade backtesting \
    --config user_data/configs/config_rl_hybrid.json \
    --strategy MtfScalper_RL_Hybrid \
    --freqaimodel MtfScalperRLModel \
    --timeframe 5m \
    --timerange 20240101-20240201
"""

print(f"🔧 دستور اجرایی:\n{train_command}")
print("\n🚀 شروع آموزش...")

!{train_command}

## 📈 مرحله ۶: اجرای بک‌تست کامل

In [ ]:
# اجرای بک‌تست برای یک ماه دیگر
print("📈 اجرای بک‌تست کامل...")

backtest_command = """freqtrade backtesting \
    --config user_data/configs/config_rl_hybrid.json \
    --strategy MtfScalper_RL_Hybrid \
    --freqaimodel MtfScalperRLModel \
    --timeframe 5m \
    --timerange 20240301-20240401 \
    --breakdown day
"""

print(f"🔧 دستور اجرایی:\n{backtest_command}")
print("\n🚀 شروع بک‌تست...")

!{backtest_command}

## 📊 مرحله ۷: تحلیل نتایج

In [ ]:
# بررسی نتایج
import os
import pandas as pd

print("📊 تحلیل نتایج:")

# بررسی فایل‌های نتایج
results_dir = "user_data/backtest_results"
if os.path.exists(results_dir):
    print(f"📁 نتایج در: {results_dir}")
    !ls -la {results_dir}
else:
    print("❌ پوشه نتایج یافت نشد")

# بررسی مدل‌های آموزش دیده
models_dir = "user_data/models"
if os.path.exists(models_dir):
    print(f"\n🤖 مدل‌های آموزش دیده در: {models_dir}")
    !ls -la {models_dir}
else:
    print("❌ پوشه مدل‌ها یافت نشد")

print("\n✅ اجرا در کولب با موفقیت انجام شد!")
print("🎉 استراتژی RL شما آماده استفاده است!")